In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import eig
import matplotlib.pyplot as plt

pi = np.pi

In [87]:
h = 1.7e-3 # thickness [m]
H = 200e-3
L = 400e-3
Iz = (h**3)/12
l_c = 40e-3
d_x = 60e-3
d_y = 40e-3
E=3.5e10
nu=0.3
rho=2400

def m_e(a, b):
    m11 =  np.array([[3454,	    922*b,	    -922*a,	    1226,	398*b,	    548*a       ],
                    [922*b,	    320*b**2,	-252*a*b,	398*b,	160*b**2,	168*a*b     ],
                    [-922*a,    -252*a*b,	320*a**2,	-548*a,	-168*a*b,	-240*a**2   ],
                    [1226,	    398*b,	    -548*a,	    3454,	922*b,	    922*a       ],
                    [398*b,	    160*b**2,	-168*a*b,	922*b,	320*b**2,	252*a*b     ],
                    [548*a,	    168*a*b,	-240*a**2,	922*a,	252*a*b,	320*a**2    ]])

    m21 =  np.array([[394,	    323*b,	    -232*a,	    1226,	548*b,	    398*a       ],
                    [-232*b,	-120*b**2,	112*a*b,	-548*b,	-240*b**2,	-168*a*b    ],
                    [232*a,	    112*a*b,	-120*a**2,	398*a,	168*a*b,	160*a**2    ],
                    [1226,	    548*b,	    -398*a,	    394,	232*b,	    232*a       ],
                    [-548*b,	-240*b**2,	168*a*b,	-232*b,	-120*b**2,	-112*a*b    ],
                    [-398*a,	-168*a*b,	160*a**2,	-232*a,	-112*a*b,	-120*a**2   ]])
    
    m22 =   np.array([[3454,	-922*b,	    922*a,	    1226,	-398*b,	    -548*a      ],
                    [-922*b,	320*b**2,	-252*a*b,	-398*b,	160*b**2,	168*a*b     ],
                    [922*a,	    -252*a*b,	320*a**2,	548*a,	-168*a*b,	-240*a**2   ],
                    [1226,	    -398*b,	    548*a,	    3454,	-922*b,	    -922*a      ],
                    [-398*b,	160*b**2,	-168*a*b,	-922*b,	320*b**2,	252*a*b     ],
                    [-548*a,	168*a*b,	-240*a**2,	-922*a,	252*a*b,	320*a**2    ]])

    m = np.concatenate((np.concatenate((m11, m21.T), axis=1),  np.concatenate((m21, m22), axis=1)), axis=0)

    return (rho*h*a*b/6300)*m

def k_e(a,b):
    alp=a/b
    bet=b/a
    I1 = np.eye(3)
    I1[0,0]=-1
    I2 = np.eye(3)
    I2[1,1]=-1
    I3 = np.eye(3)
    I3[2,2]=-1

    k11 = np.array([[4*(bet**2+alp**2)+0.4*(7-2*nu),	    2*(2*alp**2+0.2*(1+4*nu))*b,	    2*(-2*bet**2-0.2*(1-4*nu))*a    ],
                    [2*(2*alp**2+0.2*(1+4*nu))*b,	        4*(4/3*alp**2+4/15*(1-nu))*b**2,	-4*nu*a*b                       ],
                     [2*(-2*bet**2-0.2*(1-4*nu))*a,	        -4*nu*a*b,                      	4*(4/3*bet**2+1/15*(1-nu))*a**2]])

    k21 = np.array([[-(2*(2*bet**2-alp**2)+0.4*(7-2*nu)),	2*(alp**2-0.2*(1+4*nu))*b,	        2*(2*bet**2+0.2*(1-4*nu))*a     ],
                    [2*(alp**2-0.2*(1+4*nu))*b,	            4*(2/3*alp**2-4/15*(1-nu))*b**2,    0                               ],
                    [-2*(2*bet**2+0.2*(1-nu))*a,	        0,                          	    4*(4/3*bet**2-1/15*(1-nu))*a**2]])

    k31 = np.array([[-(2*(bet**2+alp**2)-0.4*(7-2*nu)),	    2*(-2*alp**2-0.2*(1-nu))*b,	        2*(-bet**2+0.2*(1+4*nu))*a      ],
                    [2*(alp**2-0.2*(1-nu))*b,	            4*(2/3*alp**2-1/15*(1-nu))*b**2,	0                               ],
                    [2*(-bet**2+0.2*(1-nu))*a,	            0,	                                4*(2/3*bet**2-4/15*(1-nu))*a**2]])

    k41 = np.array([[2*(bet**2-2*alp**2)-0.4*(7-2*nu),	    2*(-2*alp**2-0.2*(1-nu))*b,	        2*(-bet**2+0.2*(1+4*nu))*a      ],
                    [2*(2*alp**2+0.2*(1-nu))*b,	            4*(2/3*alp**2-1/15*(1-nu))*b**2,	0                               ],
                    [2*(-bet**2+0.2*(1+4*nu))*a,	        0,	                                4*(2/3*bet**2-4/15*(1-nu))*a**2]])

    k22 = I3.T @ k11 @ I3
    k32 = I3.T @ k41 @ I3
    k42 = I3.T @ k31 @ I3
    
    k33 = I1.T @ k11 @ I1
    k43 = I1.T @ k21 @ I1

    k44 = I2.T @ k11 @ I2

    aux1 = np.concatenate((k11, k21, k31, k41), axis=1)
    aux2 = np.concatenate((k21, k22, k32, k42), axis=1)
    aux3 = np.concatenate((k31, k32, k33, k43), axis=1)
    aux4 = np.concatenate((k41, k42, k43, k44), axis=1)
    k = np.concatenate((aux1, aux2, aux3, aux4), axis=0)

    
    return E*h**3/(48*(1-nu**2)*a*b)*k



In [90]:
%timeit M = m_e(0.005, 0.005)

43.6 µs ± 2.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [91]:
%timeit K = k_e(0.005, 0.005)

73.7 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
